<a href="https://colab.research.google.com/github/abhiram-ds/git-test/blob/main/NN_MNIST_DIGIT_SUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [65]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
      self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
      self.pool1 = nn.MaxPool2d(2, 2)
      self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
      self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
      self.pool2 = nn.MaxPool2d(2, 2)
      self.conv5 = nn.Conv2d(256, 512, 3)
      self.conv6 = nn.Conv2d(512, 1024, 3)
      self.conv7 = nn.Conv2d(1024, 10, 3)

      # First fully connected layer
      self.fc1 = nn.Linear(11, 5, bias=False)
      self.fc2 = nn.Linear(5, 1, bias=False)

    def forward(self, x1, x2):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x1)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)

        n1 = torch.argmax(x,dim=1).reshape(-1,1)
        n2 = torch.cat((n1,x2),dim=1)
        #n2 = n2.reshape(-1,11)
        n  = self.fc2(self.fc1(n2))

        return F.log_softmax(x), n
    
my_nn = Net()
print(my_nn)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(1024, 10, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=11, out_features=5, bias=False)
  (fc2): Linear(in_features=5, out_features=1, bias=False)
)


In [4]:
my_nn = Net().to(device)
#summary(my_nn, input_size=((20,),())

#summary(my_nn, input_size=[(1, 28, 28), (1,) ])

In [63]:
torch.manual_seed(1)
batch_size = 128

train_data = datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))

test_data = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
train_random_input = torch.randint(0,10,(60000,))
train_target2 = torch.add(train_random_input, train_data.targets)

test_random_input = torch.randint(0,10,(10000,))
test_target2 = torch.add(test_random_input, test_data.targets)
train_one_hot = torch.nn.functional.one_hot(train_random_input)
test_one_hot = torch.nn.functional.one_hot(test_random_input)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_data,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data,
    batch_size=batch_size, shuffle=True, **kwargs)

from torch.utils.data import TensorDataset

random_train_dataset = TensorDataset(train_one_hot, train_target2)
random_train_loader = torch.utils.data.DataLoader(random_train_dataset, batch_size=batch_size, shuffle=False, **kwargs)

random_test_dataset = TensorDataset(test_one_hot, test_target2)
random_test_loader = torch.utils.data.DataLoader(random_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [64]:
learning_rate = 0.01

optimizer = torch.optim.SGD(my_nn.parameters(), lr=learning_rate) 

In [53]:
# Import the tqdm function 
#from tqdm import tqdm
#predicted=torch.FloatTensor().to(device)
def train(model, device, train_loader1, train_loader2, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    #predicted_sum=torch.FloatTensor().to(device)
    for batch_idx, (dt_1, dt_2) in enumerate(zip(train_loader1, train_loader2)):
        data1, data2,target1,  target2 = dt_1[0].to(device), dt_2[0].float().to(device), dt_1[1].to(device), dt_2[1].type(torch.LongTensor).to(device)
        #print(data1.shape,target1.shape, data2.shape, target2.shape)
        optimizer.zero_grad()
        output1, output2 = model(data1, data2)
        criterion1 = nn.NLLLoss()
        loss1 = criterion1(output1, target1)
        criterion2 = nn.MSELoss()
        loss2 = criterion2(output2, target2.float())
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        #predicted_sum=torch.cat((predicted_sum,output2.detach().reshape(-1)))
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        #if batch_idx % 50 == 0:
        print(f'loss={loss.item()} batch_id={batch_idx}')
    return 0

In [ ]:
optimizer.zero_grad()
for epoch in range(1, 2):
    # Train the model using all the defined objects
    #predicted=torch.FloatTensor().to(device)
    predicted = train(my_nn, device, train_loader, random_train_loader, optimizer, epoch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss=75.78359985351562 batch_id=0
loss=82.34078979492188 batch_id=1
loss=91.12466430664062 batch_id=2
loss=77.95240783691406 batch_id=3
loss=92.49732208251953 batch_id=4
loss=82.15525817871094 batch_id=5
loss=83.57575225830078 batch_id=6
loss=92.25463104248047 batch_id=7
loss=80.9339828491211 batch_id=8
loss=91.98255920410156 batch_id=9
loss=94.4958267211914 batch_id=10
loss=78.20368957519531 batch_id=11
loss=82.64253997802734 batch_id=12
loss=92.53591918945312 batch_id=13
loss=84.12726593017578 batch_id=14
loss=77.38500213623047 batch_id=15
loss=83.34376525878906 batch_id=16
loss=82.51246643066406 batch_id=17
loss=79.1717529296875 batch_id=18
loss=79.38369750976562 batch_id=19
loss=84.0156478881836 batch_id=20
loss=85.23136138916016 batch_id=21
loss=72.80091094970703 batch_id=22
loss=86.47996520996094 batch_id=23
loss=88.51890563964844 batch_id=24
loss=88.58478546142578 batch_id=25
loss=78.21577453613281 batch_id=26
loss=86.19412231445312 batch_id=27
loss=81.30707550048828 batch_id=28

In [ ]:
test_dataset = TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
def test(model, device, test_loader):
    test_predict=torch.FloatTensor().to(device)
    my_nn.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = my_nn(data)
            test_criterion = nn.MSELoss()
            test_loss += test_criterion(output, target.float()) # sum up batch loss
            #pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            #correct += pred.eq(target.view_as(pred)).sum().item()
            test_predict=torch.cat((test_predict,output.detach().reshape(-1)))

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f} '.format(test_loss))
    return test_predict